In [16]:
#
# Importando bibliotecas necessarias
# 

import os
import re
import lxml
from collections import OrderedDict
from time import time, sleep
from datetime import datetime

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup, SoupStrainer

pd.set_option('display.max_columns', 10)

In [17]:
#
# Definindo funcoes auxiliares
#

def gera_url(ano, serie, campeonato = 'brasileiro'):
    """
    Gera url para raspagem de dados. 
    
    ano: int dentre 2010 e 2018
    serie: 'A' ou 'B'
    campeonato: 'brasileiro'
    """
    url = "https://pt.wikipedia.org/wiki/Resultados_do_primeiro_turno_do_Campeonato_Brasileiro_de_Futebol_de_" + \
    str(ano) + \
    "_-_S%C3%A9rie_" + \
    serie.upper()
    return url
    
def raspa_pagina_atual(web_driver, ano, serie, campeonato = 'brasileiro'):
    """ 
    Coleta dados sobre placares e informacoes adicionais sobre a partida da atual pagina.
    
    web_driver: selenium WebDriver em alguma pagina que contenha informacao do campeonato
    ano: int dentre 2010 e 2018
    serie: 'A' ou 'B'
    campeonato: 'brasileiro'
    """ 
    trainer = SoupStrainer('table', attrs = {'style':re.compile('width:100%;\\sbackground:\\stransparent[:alnum:]*')})
    soup = BeautifulSoup(web_driver.page_source, 'lxml-xml', parse_only = trainer)
    
    tabs_com_info = soup.find_all('table')
    
    placares = [x.find_all('tr')[0].text.split('\n') for x in tabs_com_info]
    placares = [list(filter(None, x)) for x in placares]
    placares_names = ['Data','Mandante','Placar','Visitante','Estadio']

    placares_dict = OrderedDict()
    ii = 0
    for ii, zz in enumerate(placares):
        placares_dict[ii] = dict(zip(placares_names, placares[ii]))
        ii += 1

    placares_df = pd.DataFrame.from_dict(placares_dict).transpose()

    info_adicional = [x.find_all('tr')[1] for x in tabs_com_info]
    info_adicional = pd.DataFrame({'Infos' : [x.text for x in info_adicional]})

    df_campeonato = pd.concat([placares_df, info_adicional], axis = 1)
    df_campeonato = df_campeonato.assign(Ano = ano, 
                                         Serie = serie.upper(), 
                                         Campeonato = campeonato.title())
    return df_campeonato


In [18]:
%time

#
# Coletando os dados. Preciso ter o chromedriver ou outro driver compativel com o selenium
#

wiki = webdriver.Chrome(executable_path = r'./chromedriver')
todos = pd.DataFrame()


#
# No wikipedia, ha informacao dos campeonatos brasileiros series A e B, de 2009 -> 2017.
#

series = ['a','b']
anos = range(2009, 2018)
grid = [(ano, serie) for ano in anos for serie in series]

for ano, serie in grid:
    wiki.get(gera_url(ano, serie))
    todos = todos.append(raspa_pagina_atual(wiki, ano, serie))
    
wiki.close()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [22]:
#
# Dados coletados:
#

todos = todos.reset_index().drop(columns=['index'])
todos.head()

,Data,Estadio,Mandante,Placar,Visitante,Infos,Ano,Serie,Campeonato
0,9 de maio,"Ilha do Retiro, Recife",Sport,1 – 1,Barueri,\n18:30\n\nIgor 13'\n\nRelatório\n\n 48' Pedr...,2009,A,Brasileiro
1,9 de maio,"Estádio Palestra Itália, São Paulo",Palmeiras,2 – 1,Coritiba,\n18:30\n\nWilliams 69'Keirrison 88'\n\nRela...,2009,A,Brasileiro
2,9 de maio,"Ressacada, Florianópolis",Avaí,2 – 2,Atlético-MG,\n18:30\n\nEvando 25'Muriqui 58'\n\nRelatóri...,2009,A,Brasileiro
3,10 de maio,"Pacaembu, São Paulo",Corinthians,0 – 1,Internacional,\n16:00\n\n\n\nRelatório\n\n 9' Nilmar\n\nPúbl...,2009,A,Brasileiro
4,10 de maio,"Maracanã, Rio de Janeiro",Fluminense,1 – 0,São Paulo,\n16:00\n\nMaurício 2'\n\nRelatório\n\n\n\nPú...,2009,A,Brasileiro


In [20]:
#
# Salvando para tratar a base coletada posteriormente.
#
todos.to_csv(r'./0_dados_wiki_brasileiro_raw.csv')